In [ ]:
import pandas as pd
import requests
import time
from xml.etree import ElementTree
import sys

In [ ]:
game_ratings = pd.DataFrame([])
# game_ratings = pd.read_csv('game_ratings.csv', encoding='utf-8', index_col=0)
user_games = pd.DataFrame([])

games_done = set(game_ratings.index.values)

In [ ]:
def addgame(df, game_id):
    if game_id in games_done:
        print('Skipping %d, already loaded.' % game_id)
        return
    
    games_done.add(game_id)
    
    url = "http://www.boardgamegeek.com/xmlapi/boardgame/%d?comments=1&pagesize=100" % game_id
    page = 1
    
    total = 0
    votes = 0

    
    try:
        while(True):
            sys.stdout.write('.')
            sys.stdout.flush()
    #         print('Getting page %d...' % page)
            r = requests.get('%s&page=%d' % (url,page))
            tree = ElementTree.fromstring(r.content)
            assert tree.tag == 'boardgames'

            if page == 1:
    #             print(r.content)
                print('%d is %s' % (game_id, tree.findall(".//name[@primary='true']")[0].text))

            comments = tree.findall(".//comment")
    #         print("comments ", len(comments))
    #         print()

            for comment in comments:
                if comment.get('rating') != 'N/A':
#                     print(comment.get('username'),comment.get('rating'))
                    df.loc[game_id,comment.get('username')] = float(comment.get('rating'))
                    votes = votes + 1
                    total = total + float(comment.get('rating'))

            if len(comments) != 100:
                break

            page = page + 1

        print('%d votes added.' % votes)
    except:
        print('Caught an error... :(')
        pass
    
    return df

addgame(game_ratings, 205637)

In [ ]:
def get_user(user_games, user):
    print('Getting user %s' % user)
    url = "http://www.boardgamegeek.com/xmlapi/collection/" + user
    r = requests.get(url)
    while(r.status_code == 202):
        time.sleep(1)
        print("Slept 1")
        r = requests.get(url)
        
#     print(r.content)
    tree = ElementTree.fromstring(r.content)
    
    for game in tree.findall(".//item[@subtype='boardgame']"):
        user_games.loc[user, game.get('objectid')] = 1
        
#     print(r.content)
    
    return user_games

def find_a_user(game_ratings, user_games):
    user = list(set(game_ratings.columns.values).difference(user_games.T.columns.values))[0]
    return get_user(user_games, user)

def find_game(game_ratings, user_games):
    for game_id in user_games.sum().sort_values(ascending=False).index.values:
        game_id = int(game_id)
        if not(game_id in games_done):
            print('Found %d' % game_id)
        
            return addgame(game_ratings, game_id)

    print('Getting more users')
    find_a_user(game_ratings, user_games)
    return find_game(game_ratings, user_games)
        
# find_a_user(game_ratings, user_games)


In [ ]:
addgame(game_ratings, 205637)

# print("Priming...")
# for i, user_id in enumerate(game_ratings.columns.values):
#     if i > 100:
#         break
#     requests.get("http://www.boardgamegeek.com/xmlapi/collection/%s" % user_id)
# print("Primed.")

# import numpy as np
# for i in np.arange(100 - user_games.shape[0]):
#     find_a_user(game_ratings, user_games)

while(True):
    print(find_game(game_ratings, user_games).shape)
    if game_ratings.index.values.shape[0] % 10 == 0:
        find_a_user(game_ratings, user_games)
    if game_ratings.index.values.shape[0] % 50 == 0:
        game_ratings.to_csv('game_ratings.csv', encoding='utf-8')